### Image Smilarity

###### With SIFT

In [1]:
from fastapi import FastAPI, File, UploadFile
import uvicorn
import cv2
import wget
import nest_asyncio

In [2]:
#we are not using Hamming-distance as matcher with sift because works only for binary feature-types like ORB and 
#SIFT return a detectorType() of CV32F (=float)

def sift_with_flann(img1, img2):
    img1 = cv2.cvtColor(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.cvtColor(img2, cv2.IMREAD_GRAYSCALE)
    sift = cv2.xfeatures2d.SIFT_create()
    kp_1, desc_1 = sift.detectAndCompute(img1, None)
    kp_2, desc_2 = sift.detectAndCompute(img2, None)
    index_params = dict(algorithm=0, trees=5)
    search_params = dict(checks=100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc_1, desc_2, k=2)
    good_points = []
    ratio = .7
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_points.append(m)
    return len(good_points) / len(matches)

In [3]:
def sift_with_bf(img1, img2):
    img1 = cv2.cvtColor(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.cvtColor(img2, cv2.IMREAD_GRAYSCALE)
    sift = cv2.xfeatures2d.SIFT_create()
    kp_1, desc_1 = sift.detectAndCompute(img1, None)
    kp_2, desc_2 = sift.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc_1,desc_2,k=2)
    # Apply ratio test
    good_points = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good_points.append([m])
    return len(good_points) / len(matches)

In [4]:
img111 = cv2.imread('img111.jpg')
img222 = cv2.imread('img222.jpg')

In [5]:
sift_with_flann(img111,img222)

0.34971726837755546

In [6]:
sift_with_bf(img111,img222)

0.3688560243584167

###### ORB

In [7]:
def orb_bf(img1, img2):
    img1 = cv2.imread(img1)
    img2 = cv2.imread(img2)
    img1 = cv2.cvtColor(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.cvtColor(img2, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    kp_a, desc_a = orb.detectAndCompute(img1, None)
    kp_b, desc_b = orb.detectAndCompute(img2, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True) 
    matches = bf.match(desc_a, desc_b)
    #Look for similar regions with distance < 30, as we would increase its value it precision would go down. 
    similar_regions = [i for i in matches if i.distance < 50]  
    if len(matches) == 0:
        return 0
    return len(similar_regions) / len(matches)

###### Have created the api based on orb one

In [8]:
app = FastAPI(
    title="Image Similarity API",
    description="A simple API that would return image similarity",
    version="0.1",
)

In [9]:
#@app.get("/compare")
@app.post("/compare_upload")
async def compare_images_upload(img1: UploadFile,img2: UploadFile):
    similarity=orb_bf(img1.filename,img2.filename)
    similarity_percentage = round((similarity*100),2)
    return similarity_percentage

In [10]:
#@app.get("/compare")
@app.post("/compare_url")
async def compare_images_url(img1Url: str,img2Url: str):
    filename1 = wget.download(img1Url)
    filename2 = wget.download(img2Url)
    similarity=orb_bf(filename1,filename2)
    similarity_percentage = round((similarity*100),2)
    return similarity_percentage

In [11]:
nest_asyncio.apply()

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8080, debug=True)

INFO:     Started server process [9100]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:61170 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61170 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61170 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:61171 - "POST /compare_upload HTTP/1.1" 200 OK
100% [............................................................................] 184294 / 184294INFO:     127.0.0.1:61189 - "POST /compare_url?img1Url=https%3A%2F%2Fi.ibb.co%2FdJBHXy9%2Fimg111.jpg&img2Url=https%3A%2F%2Fi.ibb.co%2Fr4SRMXG%2Fimg222.jpg HTTP/1.1" 200 OK
100% [............................................................................] 226730 / 226730INFO:     127.0.0.1:61230 - "POST /compare_url?img1Url=https%3A%2F%2Fi.ibb.co%2FdJBHXy9%2Fimg111.jpg&img2Url=https%3A%2F%2Fi.ibb.co%2FdJBHXy9%2Fimg111.jpg HTTP/1.1" 200 OK
